# 构建一个抽取链

第一部分略  跟之前的是一样的

这里是看few-shot

然后自己写的案例数据，就直接拿一个数据来

In [2]:
from datasets import load_dataset
ds = load_dataset("qgyd2021/chinese_ner_sft", "CMeEE", trust_remote_code=True) 
ds

README.md:   0%|          | 0.00/9.08k [00:00<?, ?B/s]

chinese_ner_sft.py:   0%|          | 0.00/27.3k [00:00<?, ?B/s]

cmeee.jsonl:   0%|          | 0.00/19.9M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'entities', 'data_source'],
        num_rows: 20000
    })
})

In [9]:
ds_list = ds["train"].to_list()
ds_list

[{'text': '（5）房室结消融和起搏器植入作为反复发作或难治性心房内折返性心动过速的替代疗法。',
  'entities': {'start_idx': [3, 9, 16, 35],
   'end_idx': [7, 13, 33, 38],
   'entity_text': ['房室结消融', '起搏器植入', '反复发作或难治性心房内折返性心动过速', '替代疗法'],
   'entity_label': ['pro', 'pro', 'dis', 'pro'],
   'entity_names': [['医疗程序', '医疗过程'],
    ['医疗程序', '医疗过程'],
    ['疾病', '病名', '病症'],
    ['医疗程序', '医疗过程']]},
  'data_source': 'CMeEE'},
 {'text': '（6）发作一次伴血流动力学损害的室性心动过速（ventriculartachycardia），可接受导管消融者。',
  'entities': {'start_idx': [8, 23, 50],
   'end_idx': [21, 44, 53],
   'entity_text': ['血流动力学损害的室性心动过速', 'ventriculartachycardia', '导管消融'],
   'entity_label': ['dis', 'dis', 'pro'],
   'entity_names': [['疾病', '病名', '病症'], ['疾病', '病名', '病症'], ['医疗程序', '医疗过程']]},
  'data_source': 'CMeEE'},
 {'text': '4.第三类（1）无症状性WPW综合征患者，年龄小于5岁。',
  'entities': {'start_idx': [8],
   'end_idx': [17],
   'entity_text': ['无症状性WPW综合征'],
   'entity_label': ['dis'],
   'entity_names': [['疾病', '病名', '病症']]},
  'data_source': 'CMeEE'},
 {'text': '（2）室上性心动过速可用常规抗心律失常药物控

In [11]:
ds_list = ds["train"].to_list()
data = []
label_dict = {}
for item in ds_list:
    text = item["text"]
    item = item["entities"]
    entity_text = item["entity_text"]
    entity_label = item["entity_label"]
    entity_names = item["entity_names"]
    
    # 这里是记录到底总共有多少label的
    for label, name in zip(entity_label, entity_names):
        label_dict[label] = name
    
    # 记录当前这条数据的label
    single_result = []
    for label, label_text in zip(entity_label, entity_text):
        single_result.append((label, label_text))
    data.append(
        {
            "text": text,
            "label": single_result
        }
    )
print(len(label_dict))
print(label_dict)
print(len(data))
print(data[0])
        
    

9
{'pro': ['医疗程序', '医疗过程'], 'dis': ['疾病', '病名', '病症'], 'dru': ['药物', '药品', '医药', '药剂'], 'bod': ['身体部位', '身体位置', '人体部位'], 'sym': ['症状', '发病症状', '临床表现'], 'mic': ['致病因子'], 'equ': ['医疗设备', '医疗装置', '医疗器材', '医疗器械'], 'ite': ['诊断项目', '检查项目'], 'dep': ['科室']}
20000
{'text': '（5）房室结消融和起搏器植入作为反复发作或难治性心房内折返性心动过速的替代疗法。', 'label': [('pro', '房室结消融'), ('pro', '起搏器植入'), ('dis', '反复发作或难治性心房内折返性心动过速'), ('pro', '替代疗法')]}


In [13]:
import random
random.seed(42)
sampled_data = random.sample(data, k=5)
sampled_data

[{'text': '瑞氏综合征@【治疗】本病预后不良，早期认识轻症患儿并给予及时治疗是争取改善预后的关键。',
  'label': [('dis', '瑞氏综合征')]},
 {'text': '2.连续呼吸的Astograph法原理是应用强迫振荡技术进行检测，在连续吸入不同浓度的乙酰甲胆碱时，不断监测呼吸阻力。',
  'label': [('pro', '连续呼吸的Astograph法'),
   ('pro', '强迫振荡技术'),
   ('dru', '乙酰甲胆碱'),
   ('ite', '呼吸阻力')]},
 {'text': '偶见堵塞装置脱落，多由于操作不当或选择堵塞装置太小所致，可用异物钳抓取，必要时需外科手术。',
  'label': [('pro', '外科手术')]},
 {'text': 'IDA患儿还可以出现免疫功能低下，容易合并感染。',
  'label': [('dis', 'IDA'), ('dis', '免疫功能低下'), ('dis', '感染')]},
 {'text': '四、治疗ICH治疗原则是迅速控制出血，适时进行外科手术治疗，预防并发症与后遗症。',
  'label': [('dis', 'ICH'), ('pro', '迅速控制出血'), ('pro', '外科手术治疗')]}]

In [15]:
for k, v in label_dict.items():
    print(k, v)

pro ['医疗程序', '医疗过程']
dis ['疾病', '病名', '病症']
dru ['药物', '药品', '医药', '药剂']
bod ['身体部位', '身体位置', '人体部位']
sym ['症状', '发病症状', '临床表现']
mic ['致病因子']
equ ['医疗设备', '医疗装置', '医疗器材', '医疗器械']
ite ['诊断项目', '检查项目']
dep ['科室']


In [31]:
from pydantic import BaseModel, Field
from langchain_openai.chat_models import ChatOpenAI

class MedicalEntity(BaseModel):
    """
    提取的实体信息字段
    """
    
    pro: list[str] = Field(None, description="信息中的实体关于(医疗程序, 医疗过程)")
    dis: list[str] = Field(None, description="信息中的实体关于(疾病, 病名, 病症)")
    dru: list[str] = Field(None, description="信息中的实体关于(药物, 药品, 医药, 药剂)")
    bod: list[str] = Field(None, description="信息中的实体关于(身体部位, 身体位置, 人体部位)")
    sym: list[str] = Field(None, description="信息中的实体关于(症状, 发病症状, 临床表现)")
    mic: list[str] = Field(None, description="信息中的实体关于(致病因子)")
    equ: list[str] = Field(None, description="信息中的实体关于(医疗设备, 医疗装置, 医疗器材, 医疗器械)")
    ite: list[str] = Field(None, description="信息中的实体关于(诊断项目, 检查项目)")
    dep: list[str] = Field(None, description="信息中的实体关于(科室)")
    
llm = ChatOpenAI(
    base_url="http://localhost:8000/v1",
    api_key="empty",
    model="Qwen/Qwen2.5-7B-Instruct"
)

In [32]:
tools_llm = llm.bind_tools(tools=[MedicalEntity], temperature=0)
for item in sampled_data:
    text = item["text"]
    label = item["label"]
    response = tools_llm.invoke(text)
    print(response.additional_kwargs["tool_calls"][0]["function"]["arguments"])
    print(label)

{"pro": ["治疗"], "dis": ["瑞氏综合征"], "sym": [], "bod": [], "dru": [], "mic": [], "equ": [], "ite": [], "dep": []}
[('dis', '瑞氏综合征')]
{"pro": ["连续呼吸的Astograph法"], "dru": ["乙酰甲胆碱"], "equ": ["强迫振荡技术"]}
[('pro', '连续呼吸的Astograph法'), ('pro', '强迫振荡技术'), ('dru', '乙酰甲胆碱'), ('ite', '呼吸阻力')]
{"pro": ["操作不当", "选择堵塞装置太小"], "equ": ["堵塞装置", "异物钳"], "dep": ["外科"]}
[('pro', '外科手术')]
{"pro": [], "dis": ["IDA患儿"], "dru": [], "bod": [], "sym": ["免疫功能低下", "容易合并感染"], "mic": [], "equ": [], "ite": [], "dep": []}
[('dis', 'IDA'), ('dis', '免疫功能低下'), ('dis', '感染')]
{"pro": ["治疗"], "dis": [], "dru": [], "bod": [], "sym": [], "mic": [], "equ": [], "ite": [], "dep": []}
[('dis', 'ICH'), ('pro', '迅速控制出血'), ('pro', '外科手术治疗')]


In [40]:
from collections import defaultdict
from langchain_core.utils.function_calling import tool_example_to_messages

few_shot = sampled_data[:-1]
test_data = sampled_data[-1]

examples = []
for item in few_shot:
    text = item["text"]
    label = item["label"]
    
    medical_entity = defaultdict(list)
    for k, v in label:
        medical_entity[k].append(v)
    medical_entity = MedicalEntity(**medical_entity)
    message = tool_example_to_messages(text, [medical_entity])
    examples.extend(message)  # 这里用extend的原因是上面这个函数本来就会返回一个list

for message in examples:
    message.pretty_print()

================================ Human Message =================================

瑞氏综合征@【治疗】本病预后不良，早期认识轻症患儿并给予及时治疗是争取改善预后的关键。
================================== Ai Message ==================================
Tool Calls:
  MedicalEntity (215a0af4-1847-4a1d-96a7-5674c36aa4c3)
 Call ID: 215a0af4-1847-4a1d-96a7-5674c36aa4c3
  Args:
    pro: None
    dis: ['瑞氏综合征']
    dru: None
    bod: None
    sym: None
    mic: None
    equ: None
    ite: None
    dep: None
================================= Tool Message =================================

You have correctly called this tool.
================================ Human Message =================================

2.连续呼吸的Astograph法原理是应用强迫振荡技术进行检测，在连续吸入不同浓度的乙酰甲胆碱时，不断监测呼吸阻力。
================================== Ai Message ==================================
Tool Calls:
  MedicalEntity (b73ac415-a973-4e5e-ac74-60786a0b11db)
 Call ID: b73ac415-a973-4e5e-ac74-60786a0b11db
  Args:
    pro: ['连续呼吸的Astograph法', '强迫振荡技术']
    dis: None
    dru: ['乙酰甲胆碱']
    

In [44]:
from langchain_core.messages import HumanMessage
text = test_data["text"]
examples.append(HumanMessage(text))


print(tools_llm.invoke(examples).additional_kwargs["tool_calls"][0]["function"]["arguments"])
print(test_data["label"])

{"pro": ["外科手术治疗"], "dis": ["ICH"], "dru": null, "bod": null, "sym": null, "mic": null, "equ": null, "ite": null, "dep": null}
[('dis', 'ICH'), ('pro', '迅速控制出血'), ('pro', '外科手术治疗')]
